## (1) Design an MLP model to distinguish two black-and-white number images of  28×28  size under the condition that the number of parameters in your NN should be less than  1𝑀 . Then, train it with MNIST dataset (number 3 and 8) by using the minibatch SGD under the condition that batch size is a power of 2 less than 1024.

In [6]:
#LR은 아웃풋 1개. 수업 실습시간에 쓴 코드 쓸 것.
#아래 코드는 SOFTMAX의 경우 2개. 이 테스트 코드 쓸 것.

def test(DATASET, MODEL, train=True):
    global batch_size
    image_size = 28*28
    #image_size = 3*32*32 # for CIFAR dataset
    MLP_model = MODEL
    
    if train:
        ''' Test for train_dataset '''
        train_loader = DataLoader(dataset=DATASET, batch_size=batch_size, shuffle=False)
        count = 0
        for image,label in train_loader :
            label = label
            prediction = torch.argmax(MLP_model(image.view(len(image),image_size)),dim=1)
            count += (prediction == label).sum()

        print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

    else:
        ''' Test for test_dataset '''
        count = 0
        wrong_answer = []
        
        test_loader = DataLoader(dataset=DATASET, batch_size=batch_size, shuffle=False)
        for image,label in test_loader :
            label = label
            prediction = torch.argmax(MLP_model(image.view(len(image),image_size)),dim=1)
            count += (prediction == label).sum()

        #     else:
        #         wrong_answer += [i]   

        print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )

def test_1(DATASET, MODEL, train=True):
    LR_model = MODEL
    
    if train:
        
        ''' Test for train_dataset '''
        count = 0
        for i in range(DATASET.targets.size()[0]):
            image = DATASET.data[i]
            label = DATASET.targets[i]    

            prediction = LR_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
            if prediction == label: 
                count += 1

        print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(train_dataset.targets.size()[0], count, count/train_dataset.targets.size()[0] * 100. ) )    
    
    else:

        ''' Test for test_dataset '''
        count = 0
        wrong_answer = []
        for i in range(DATASET.targets.size()[0]):
            image = DATASET.data[i]
            label = DATASET.targets[i]    

            prediction = LR_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
            if prediction == label: 
                count += 1
            else:
                wrong_answer += [i]   

        print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(test_dataset.targets.size()[0], count, count/test_dataset.targets.size()[0] * 100. ) )

def data_preprocessing(nb1, nb2):
    '''
    1 iteration
    Shuffle = False
    '''
    from torchvision import datasets
    import torchvision.transforms as transforms

    ### dataset for training parameters
    train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
    ### dataset for testing the accuracy of the trained parameters
    test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor())

    '''
    normalizing data is not necessary if we use DataLoader
    '''
    ### normalizing dataset 
    # train_dataset.data = (train_dataset.data/255-0.1307)/0.3081
    # test_dataset.data = (test_dataset.data/255-0.1307)/0.3081

    '''
    Choose two numbers and prepare datasets with two numbers.
    '''

    train_dataset.data = torch.cat([train_dataset.data[(train_dataset.targets == nb1)],train_dataset.data[(train_dataset.targets == nb2)]],dim=0)
    train_dataset.targets = torch.cat([train_dataset.targets[(train_dataset.targets == nb1)],train_dataset.targets[(train_dataset.targets == nb2)]],dim=0)

    train_dataset.targets[(train_dataset.targets == nb1)] = 0
    train_dataset.targets[(train_dataset.targets == nb2)] = 1

    '''
    Do the same thing for the test dataset
    '''

    test_dataset.data = torch.cat([test_dataset.data[(test_dataset.targets == nb1)],test_dataset.data[(test_dataset.targets == nb2)]],dim=0)
    test_dataset.targets = torch.cat([test_dataset.targets[(test_dataset.targets == nb1)],test_dataset.targets[(test_dataset.targets == nb2)]],dim=0)

    test_dataset.targets[(test_dataset.targets == nb1)] = 0
    test_dataset.targets[(test_dataset.targets == nb2)] = 1
    
    return train_dataset, test_dataset

In [42]:
'''
Use nn.Linear and nn.Sigmoid
'''


import torch
import torch.nn as nn
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html

'''
Using GPU
'''
import torch
import torch.optim as optim # See https://pytorch.org/docs/stable/optim.html
from utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

class MLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=400) :
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(self.in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True)
        self.act2 = nn.Sigmoid()
        self.linear3 = nn.Linear(inter_dim, inter_dim, bias=True)                
        self.act3 = nn.Sigmoid()
        self.linear4 = nn.Linear(inter_dim, inter_dim, bias=True)                
        self.act4 = nn.Sigmoid()
        self.linear5 = nn.Linear(inter_dim, inter_dim, bias=True)                
        self.act5 = nn.Sigmoid()
        self.linear6 = nn.Linear(inter_dim, out_dim, bias=True)                
        self.act6 = nn.Sigmoid()
        
    def forward(self, x) :
#         assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        z = self.linear1(x.view(-1, 28*28))
        z = self.act1(z)
        #z = self.linear2(z)   
        #z = self.act2(z)
        #z = self.linear3(z)   
        #z = self.act3(z)
        #z = self.linear4(z)   
        #z = self.act4(z)
        #z = self.linear5(z)   
        #z = self.act5(z)
        z = self.linear6(z)   
        z = self.act6(z)
        return z

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from utils import *

'''
DataLoader is used to apply minibatch SGD
'''
train_dataset, test_dataset = data_preprocessing(3,8)
batch_size = 256
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

''' Define model, optimizer and loss function'''
MLP_model = MLP(28*28,1).to(device)
OPTIMIZER = optim.SGD(MLP_model.parameters(),  lr=0.1)
loss = torch.nn.BCELoss()


import time
start = time.time()

epochs = 200

for i in range(epochs):
    MLP_model.to(device)
    for image,label in train_loader :

        # Forward processeeebebb
        hat_y = MLP_model(image.to(device).view(-1, 28*28)).to(device)
        cost = loss(hat_y.reshape([hat_y.size()[0]]), label.float().to(device))

        # Wipe up gradient in the previous step
        OPTIMIZER.zero_grad() 

        # Computes the gradient of current tensor w.r.t. graph leaves.
        cost.backward() 

        # Updates the parameters
        OPTIMIZER.step()
    print(f"{i}-epoch/total {epochs}", end = " ")
    test_1(train_dataset, MLP_model.to("cpu"))
end = time.time()
print("Training Time: {:.2f}".format(end-start))
        
test_1(test_dataset, MLP_model.to("cpu"), train=False)

total_params = sum(p.numel() for p in MLP_model.parameters())
print("Number of Parameters: ", total_params)

0-epoch/total 200 total: 11982, correctly matched: 10618 accuracy: 88.62%

1-epoch/total 200 total: 11982, correctly matched: 10758 accuracy: 89.78%

2-epoch/total 200 total: 11982, correctly matched: 10939 accuracy: 91.30%

3-epoch/total 200 total: 11982, correctly matched: 11040 accuracy: 92.14%

4-epoch/total 200 total: 11982, correctly matched: 11122 accuracy: 92.82%

5-epoch/total 200 total: 11982, correctly matched: 11196 accuracy: 93.44%

6-epoch/total 200 total: 11982, correctly matched: 11171 accuracy: 93.23%

7-epoch/total 200 total: 11982, correctly matched: 11198 accuracy: 93.46%

8-epoch/total 200 total: 11982, correctly matched: 11181 accuracy: 93.31%

9-epoch/total 200 total: 11982, correctly matched: 11188 accuracy: 93.37%

10-epoch/total 200 total: 11982, correctly matched: 11209 accuracy: 93.55%

11-epoch/total 200 total: 11982, correctly matched: 11180 accuracy: 93.31%

12-epoch/total 200 total: 11982, correctly matched: 11222 accuracy: 93.66%

13-epoch/total 200 tot

108-epoch/total 200 total: 11982, correctly matched: 11438 accuracy: 95.46%

109-epoch/total 200 total: 11982, correctly matched: 11398 accuracy: 95.13%

110-epoch/total 200 total: 11982, correctly matched: 11451 accuracy: 95.57%

111-epoch/total 200 total: 11982, correctly matched: 11423 accuracy: 95.33%

112-epoch/total 200 total: 11982, correctly matched: 11472 accuracy: 95.74%

113-epoch/total 200 total: 11982, correctly matched: 11470 accuracy: 95.73%

114-epoch/total 200 total: 11982, correctly matched: 11456 accuracy: 95.61%

115-epoch/total 200 total: 11982, correctly matched: 11438 accuracy: 95.46%

116-epoch/total 200 total: 11982, correctly matched: 11466 accuracy: 95.69%

117-epoch/total 200 total: 11982, correctly matched: 11476 accuracy: 95.78%

118-epoch/total 200 total: 11982, correctly matched: 11476 accuracy: 95.78%

119-epoch/total 200 total: 11982, correctly matched: 11476 accuracy: 95.78%

120-epoch/total 200 total: 11982, correctly matched: 11512 accuracy: 96.08%


# --- Test Code ---

In [43]:
DATASET = test_dataset
LR_model = MLP_model

''' Test for test_dataset '''
count = 0
wrong_answer = []
for i in range(DATASET.targets.size()[0]):
    image = DATASET.data[i]
    label = DATASET.targets[i]    

    prediction = LR_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
    if prediction == label: 
        count += 1
    else:
        wrong_answer += [i]   

print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(test_dataset.targets.size()[0], count, count/test_dataset.targets.size()[0] * 100. ) )

total: 1984, correctly matched: 1912 accuracy: 96.37%



# --- Review ---

### layer 수를 3개 이상으로 올리니 정확도 내려감.
### batch_size를 256이상으로 올리니 정확도 내려감.
### (batch_size를 올린 만큼 epoch 수를 올려야 하지만 학습 시간이 오래걸려 test 불가)
### batch = 256, epoch = 200으로 마무리
### inter_dim 400이상은 Parameter 1M 제한을 넘어감.
### inter_dim 400미만은 정확도 내려감.
